<a href="https://colab.research.google.com/github/Tasnima158/clip/blob/main/CLIP_chest_xray_pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"tasnimaahmed","key":"8dba801f027d63f06be93c0f4d59fb00"}'}

In [ ]:
!ls /content


'kaggle (1).json'   kaggle.json   sample_data


In [ ]:
!pip install kagglehub


In [ ]:
!ls $path


chest_xray


In [ ]:
!ls $path/chest_xray


chest_xray  __MACOSX  test  train  val


In [ ]:
!ls $path/chest_xray/train


NORMAL	PNEUMONIA


In [ ]:
import kagglehub

path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'chest-xray-pneumonia' dataset.
Path to dataset files: /kaggle/input/chest-xray-pneumonia


# STEP 1: Install CLIP

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-so51us0y
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-so51us0y
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


# STEP 2: Import libraries & load pretrained CLIP

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torchvision.datasets import ImageFolder

import clip
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"

model, preprocess = clip.load("ViT-B/32", device=device)
print("CLIP loaded on:", device)


CLIP loaded on: cuda


# STEP 3: Define ZERO-SHOT text prompts
These are descriptions

NOT examples

NOT labels used for training

In [ ]:
class_prompts = [
    "a chest X-ray showing pneumonia",
    "a normal chest X-ray"
]

text_tokens = clip.tokenize(class_prompts).to(device)


# STEP 4: Load TEST images (labels used ONLY for evaluation)

In [ ]:
test_dir = os.path.join(path, "chest_xray", "test")
image_paths = []
true_labels = []

for label in ["PNEUMONIA", "NORMAL"]:
    folder = os.path.join(test_dir, label)

    for img in os.listdir(folder):
        image_paths.append(os.path.join(folder, img))
        true_labels.append(label)

print("Total test images:", len(image_paths))
pint(" total lebels :" , len (true_labels))


Total test images: 624


In [ ]:
print(image_paths)

['/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person1676_virus_2892.jpeg', '/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person1650_virus_2852.jpeg', '/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person22_virus_55.jpeg', '/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person122_bacteria_582.jpeg', '/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person85_bacteria_417.jpeg', '/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person79_virus_148.jpeg', '/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person29_virus_64.jpeg', '/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person123_bacteria_587.jpeg', '/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person75_virus_136.jpeg', '/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person153_bacteria_725.jpeg', '/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person173_bacteria_831.jpeg', '/kaggle

In [ ]:
print(true_labels)

['PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA'

In [ ]:
correct = 0

with torch.no_grad():
    text_features = model.encode_text(text_tokens)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    for img_path, true_label in zip(image_paths, true_labels):
        image = preprocess(Image.open(img_path).convert("RGB")) \
                .unsqueeze(0).to(device)

        image_features = model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        similarity = (image_features @ text_features.T).softmax(dim=-1)
        pred_index = similarity.argmax().item()

        predicted_label = "PNEUMONIA" if pred_index == 0 else "NORMAL"

        if predicted_label == true_label:
            correct += 1

zero_shot_accuracy = correct / len(image_paths)
zero_shot_accuracy


0.625

In [ ]:
print((image_features @ text_features.T))

tensor([[0.3579, 0.3192]])


In [ ]:
print((similarity.shape))

torch.Size([1, 2])


In [ ]:
print(text_features.shape)

torch.Size([2, 512])


In [ ]:
print(text_features)

tensor([[-0.0280,  0.0441, -0.0180,  ...,  0.0028, -0.0178, -0.0218],
        [ 0.0090,  0.0207, -0.0429,  ..., -0.0323, -0.0021, -0.0163]])


In [ ]:
print(image_features.shape)

torch.Size([1, 512])


In [ ]:
print(image_features)

tensor([[ 1.8593e-02, -1.8842e-02, -2.6724e-02,  9.9465e-03, -5.7080e-03,
          4.1310e-02,  3.2511e-02,  1.4482e-02,  8.2266e-02, -1.6523e-02,
          8.2524e-02, -3.9324e-02,  4.5027e-02,  2.1937e-02,  3.8956e-02,
         -2.9245e-02,  8.1033e-02, -2.5897e-02, -2.1710e-02,  1.2925e-03,
         -3.2093e-02,  8.2228e-03,  6.1167e-02,  4.9228e-02, -1.3044e-02,
         -3.5185e-02, -1.3705e-02,  1.1077e-02, -6.2858e-03, -4.5367e-03,
         -4.4496e-02,  3.7622e-02,  1.3855e-02,  6.6967e-04,  1.3042e-02,
         -4.7528e-02,  4.1489e-02, -5.1924e-02, -3.9428e-02, -6.9954e-03,
         -4.9145e-02, -1.7894e-02, -1.9405e-02, -6.7257e-02,  3.3869e-02,
         -1.2152e-01, -7.8145e-02,  4.5189e-02, -1.4142e-02, -4.7415e-02,
          4.8096e-02, -3.2300e-02, -1.0664e-02,  7.2389e-03, -3.5272e-03,
         -4.3502e-03, -1.1616e-03, -3.9529e-02,  2.5331e-02,  3.4214e-02,
          1.3275e-01,  2.2785e-02,  2.0081e-02, -2.1203e-02, -8.2429e-02,
         -2.5811e-02,  9.2977e-03,  5.

# FINE-TUNING CLIP (CUSTOM MODEL)

# Text descriptions

In [ ]:
# class_text = {
#     "PNEUMONIA": [
#         "a chest X-ray showing pneumonia",
#         "a chest X-ray with lung infection",
#         "a chest X-ray with abnormal lung opacity"
#     ],
#     "NORMAL": [
#         "a normal chest X-ray",
#         "a healthy chest X-ray",
#         "a chest X-ray with clear lungs"
#     ]
# }


In [ ]:
print(Dataset)


# Create a PyTorch Dataset

In [ ]:
class ChestXrayDataset(Dataset):
    def __init__(self, root_dir):
        self.samples = []
        for label in ["PNEUMONIA", "NORMAL"]:
            folder = os.path.join(root_dir, label)
            for img in os.listdir(folder):
                self.samples.append((os.path.join(folder, img), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = preprocess(Image.open(img_path).convert("RGB"))

        label_id = 0 if label == "PNEUMONIA" else 1
        return image, label_id


# Create DataLoader

In [ ]:
train_dir = os.path.join(path, "chest_xray", "train")
test_dir  = os.path.join(path, "chest_xray", "test")

train_dataset = ChestXrayDataset(train_dir)
test_dataset  = ChestXrayDataset(test_dir)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Train samples:", len(train_dataset))
print("Test samples:", len(test_dataset))


Train samples: 5216
Test samples: 624


# Freeze most of CLIP

In [ ]:
for param in model.parameters():
    param.requires_grad = False

for param in model.visual.parameters():
    param.requires_grad = True


In [ ]:
classifier = torch.nn.Linear(512, 2).to(device)


# Optimizer & loss

In [ ]:
optimizer = torch.optim.AdamW(
    list(model.visual.parameters()) + list(classifier.parameters()),
    lr=3e-5
)

loss_fn = torch.nn.CrossEntropyLoss()


# Fine-tuning loop

In [ ]:
model.train()
classifier.train()

epochs = 5

for epoch in range(epochs):
    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        with torch.cuda.amp.autocast(False):
            image_features = model.encode_image(images)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            image_features = image_features.float()

        logits = classifier(image_features)
        loss = loss_fn(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


/tmp/ipython-input-2498660111.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(False):


Epoch 1, Loss: nan
Epoch 2, Loss: nan
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan


# Evaluate fine-tuned CLIP

In [ ]:
model.eval()
classifier.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        image_features = model.encode_image(images)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        image_features = image_features.float()

        logits = classifier(image_features)
        preds = logits.argmax(dim=1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

finetuned_accuracy = correct / total
finetuned_accuracy


0.625

# ImageNet-CNN_DenseNet-121

In [ ]:
cnn_train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

cnn_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
cnn_train_dataset = ImageFolder(
    root=os.path.join(path, "chest_xray", "train"),
    transform=cnn_train_transform
)

cnn_test_dataset = ImageFolder(
    root=os.path.join(path, "chest_xray", "test"),
    transform=cnn_test_transform
)

cnn_train_loader = DataLoader(
    cnn_train_dataset, batch_size=32, shuffle=True
)

cnn_test_loader = DataLoader(
    cnn_test_dataset, batch_size=32, shuffle=False
)

print("CNN classes:", cnn_train_dataset.classes)


CNN classes: ['NORMAL', 'PNEUMONIA']


In [ ]:
cnn_model = models.densenet121(pretrained=True)

cnn_model.classifier = nn.Linear(
    cnn_model.classifier.in_features, 2
)

cnn_model = cnn_model.to(device)


In [ ]:
cnn_criterion = nn.CrossEntropyLoss()

cnn_optimizer = torch.optim.Adam( cnn_model.parameters(), lr=1e-4)


In [ ]:
cnn_model.train()
epochs = 5

for epoch in range(epochs):
    total_loss = 0

    for images, labels in cnn_train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = cnn_model(images)
        loss = cnn_criterion(outputs, labels)

        cnn_optimizer.zero_grad()
        loss.backward()
        cnn_optimizer.step()

        total_loss += loss.item()

    print(f"[CNN] Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(cnn_train_loader):.4f}")


[CNN] Epoch 1/5, Loss: 0.0930
[CNN] Epoch 2/5, Loss: 0.0385
[CNN] Epoch 3/5, Loss: 0.0207
[CNN] Epoch 4/5, Loss: 0.0163
[CNN] Epoch 5/5, Loss: 0.0090


In [ ]:
cnn_model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in cnn_test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = cnn_model(images)
        preds = outputs.argmax(dim=1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

cnn_accuracy = correct / total
cnn_accuracy


0.8205128205128205

In [ ]:
print(f"Zero-shot CLIP Accuracy     : {zero_shot_accuracy:.4f}")
print(f"Fine-tuned CLIP Accuracy    : {finetuned_accuracy:.4f}")
print(f"ImageNet CNN Accuracy       : {cnn_accuracy:.4f}")


Zero-shot CLIP Accuracy     : 0.6250
Fine-tuned CLIP Accuracy    : 0.6250
ImageNet CNN Accuracy       : 0.8205


# FINE TUNED CNN

In [ ]:
cnn_frozen = models.densenet121(pretrained=True)
cnn_frozen.classifier = nn.Linear(
    cnn_frozen.classifier.in_features, 2
)
cnn_frozen = cnn_frozen.to(device)


In [ ]:
for param in cnn_frozen.features.parameters():
    param.requires_grad = False


In [ ]:
frozen_optimizer = torch.optim.Adam(
    cnn_frozen.classifier.parameters(),
    lr=1e-3
)
criterion = nn.CrossEntropyLoss()


In [ ]:
cnn_frozen.train()
epochs = 5

for epoch in range(epochs):
    total_loss = 0

    for images, labels in cnn_train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = cnn_frozen(images)
        loss = criterion(outputs, labels)

        frozen_optimizer.zero_grad()
        loss.backward()
        frozen_optimizer.step()

        total_loss += loss.item()

    print(f"[Frozen CNN] Epoch {epoch+1}, Loss: {total_loss/len(cnn_train_loader):.4f}")


[Frozen CNN] Epoch 1, Loss: 0.2840
[Frozen CNN] Epoch 2, Loss: 0.1721
[Frozen CNN] Epoch 3, Loss: 0.1466
[Frozen CNN] Epoch 4, Loss: 0.1319
[Frozen CNN] Epoch 5, Loss: 0.1310


In [ ]:
cnn_frozen.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in cnn_test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = cnn_frozen(images)
        preds = outputs.argmax(dim=1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

cnn_frozen_accuracy = correct / total
cnn_frozen_accuracy


0.8926282051282052